In [1]:
import os
import logging
import boto3
import zipfile
import json
import io
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from io import BytesIO

In [2]:
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3', 
                  endpoint_url=s3_host, 
                  aws_access_key_id=access_key_id, 
                  aws_secret_access_key=secret_access_key)

bucket = 'twitter.ei'


In [3]:
def process(file):
    zip_object = s3.get_object(Bucket=bucket, Key=file)
    zip_contents = zip_object['Body'].read()
    zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
    for file_info in zip_file.infolist():
        with zip_file.open(file_info) as csv_file:
            df = None
            try:
                if df == None:
                    df = pd.read_csv(csv_file)
                else:
                    df = pd.concat([df, pd.read_csv(csv_file)], axis=0)
            except:
                print(f"{csv_file.filename} is not a csv file")
    return df


In [4]:
## used to construct userid, twitterid dictionary
## not actually needed for retweet

def add_to_userdict(row, user_dict):
    userid = row['userid']
    twitterid = row['user_screen_name']
    ## since user_mentions only has userid, we cannot obtain the twitterids with user_mentions in twitterei
    if userid not in user_dict:
        user_dict[userid] = [twitterid]
    elif twitterid not in user_dict[userid]:
        print(f"{twitterid} has multiple twitterids")
        user_dict[userid].append(twitterid)


In [11]:
## this method should turn the string back into a list
def clean (str):
    str = str[1:-1]
    return str.split(',')

def count_mentioned_users(row, mentioned_dict):
    user_mentions = row['user_mentions']
    if not pd.isna(user_mentions):
        if not isinstance(user_mentions, list):
            user_mentions = clean(user_mentions)
        for user in user_mentions:
            mentioned_dict[user] += 1

In [12]:
folder_prefix_lst = ["2019_08/china_082019_1/china_082019_1_tweets_csv_unhashed.zip",
                     "2019_08/china_082019_2/china_082019_2_tweets_csv_unhashed.zip",
                     "2019_08/china_082019_3/china_082019_3_tweets_csv_unhashed.zip",
                     "2020_05/china_052020/china_052020_tweets_csv_unhashed.zip", 
                     "2020_09/thailand_092020/thailand_092020_tweets_csv_unhashed.zip"
                     ]

retweet_lst = []
mentioned_lst = []
user_dict = dict()
pd.set_option('display.float_format', lambda x: '%.0f' % x)

for folder in folder_prefix_lst:
    mentioned_dict = defaultdict(int)
    print(folder)
    data = process(folder)
    data.apply(lambda row: add_to_userdict(row, user_dict), axis = 1)
    data.apply(lambda row: count_mentioned_users(row, mentioned_dict), axis = 1)
    retweet_counts = data['retweet_userid'].value_counts()
    retweet_counts.index.astype(str)
    retweet_lst.append(retweet_counts)
    mentioned_lst.append(mentioned_dict)


2019_08/china_082019_1/china_082019_1_tweets_csv_unhashed.zip


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_66062/130882326.py:10: DtypeWarning: Columns (6,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


2019_08/china_082019_2/china_082019_2_tweets_csv_unhashed.zip


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_66062/130882326.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


2019_08/china_082019_3/china_082019_3_tweets_csv_unhashed.zip


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_66062/130882326.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_66062/130882326.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


2020_05/china_052020/china_052020_tweets_csv_unhashed.zip
2020_09/thailand_092020/thailand_092020_tweets_csv_unhashed.zip


In [16]:
title = ["china_082019_1", "china_082019_2", "china_082019_3", "china_052020", "thailand_092020"]
i = 0
for df in retweet_lst:
    df.index = df.index.astype(int)

    df.to_csv(f"~/Coding/buntain/retweet_data/twitterrei_{title[i]}.csv")
    i += 1
i = 0
for dict in mentioned_lst:
    del dict['']
    df = pd.DataFrame(dict.items(), columns = ['mentioned_userid', 'count'])
    df.to_csv(f"~/Coding/buntain/mentioned_data/twitterrei_{title[i]}.csv", index=False)
    i += 1

In [ ]:
twitterid_df = pd.DataFrame(user_dict.items(), columns = ["userid", "twitterid"])
# twitterid_df
twitterid_df.to_csv("~/Coding/buntain/retweet_data/twitterei_user_dict.csv", index=False)